# 1. Image Processing
## 1.1 Introduction

The primary goal of adaptive optics (AO) is to correct for atmospheric turbulence in real time, allowing astronomers to obtain images that approach the diffraction limit of the telescope. While AO can greatly improve image sharpness, several **post-processing** steps are still required to optimise the scientific return. 

Typically in observing campaigns, before doing science on the collected image, the series of raw data are reduced using *auxiliary* images. These are calibration images that are needed to calibrate cientific image (with photometry o spectroscopy) to correct for instrumental effects.

These *auxiliary* images are mainly of 3 types:

- **Bias images**: these are taken with zero exposure time and a closed shutter (no incoming photons). They measure the electronic offset introduced by the CCD readout process.

- **Flat field images**: these are obtained by imaging a uniformly illuminated source and allow us to measure the spatial variation in the response of the optical system and detector.

- **Dark images**: these are acquired with the same exposure time of the science image but with the shutter closed and used to remove dark current from the detector, which is a contributor to the thermal noise signal that needs to be considered, especially when our target is a faint object. Typically, these frames are neglected in the image reduction routine, as detectors are cooled to cryogenic temperatures.

So as shown in the following scheme, the processing of the collected raw images $S_{raw}$ can be expressed by the following relation:

$$
 S_{red} = \frac{S_{raw} - B}{F - B} 
$$ 

where $S_{red}$ is the science image after the reduction of the bias, $B$, and the normalization to the flat field, $F$.

<div>
<img src="correction of science image.png" width="600"/>
</div>


However, in the context of our current dataset, we will focus specifically on **background subtraction** from the collected images, without the use of *auxiliary* calibration images. This approach allows us to focus on removing unwanted background noise to enhance the signal of astronomical objects without applying full instrumental calibration.

If you want learn more about image processing techinques in see [Chap.6 Observing and calibration strategies (J. Heidt, 2022)](https://ui.adsabs.harvard.edu/abs/2022ASSL..467.....H/abstract).

So at the end of this section, we will have a cleaned up closed loop data cube image from detector noise and sky background.

## 1.2 Background Subtraction

The **background noise** comes from various sources such as :

1. sky brightness, like sky-background or other sources in the field close to the scientific target;

2. detector noise and artifacts, some pixel of the camera may have larger readout noise than others 

3. unwanted signals from the telescope (tracking drifts, ghosts from anti-reflective coatings on the optics, static aberrations, among others ...).

In the following tutorial, we propose a simple background subtraction of open and closed loop data collected on *Beta Pegasi*, with PAPYRUS NIR  imager camera [C-RED3](https://andor.oxinst.com/products/c-red-series/c-red-3) camera.

Let's start by loading the backup data and importing the needed modules.

In [2]:
!pip uninstall -y oao24
!pip install git+https://github.com/ArcetriAdaptiveOptics/OAO24.git

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from oao24 import package_data

The `InfraredExampleData` class defined in 'package_data.py' is used to load the backup files, where:

- `get_open_loop_data_cube()`: Loads and returns the open loop data cube from the file `"ID_105.npy"`.

- `get_camera_dark_data()`: Loads and returns a background region from the file `"ID_110.npy"`.

- `get_close_loop_data_cube()`: Loads and returns the closed loop data cube from the file `"ID_109_red_16.npy"`.

In [4]:
class InfraredExampleData():
    
    @staticmethod
    def get_open_loop_data_cube():
        image =  np.load(package_data.tuto3_folder() / "ID_105.npy")
        return np.atleast_3d(image)
    
    @staticmethod
    def get_camera_dark_data():
        return np.load(package_data.tuto3_folder() / "ID_110.npy")
    
    @staticmethod
    def get_close_loop_data_cube():
        return np.load(package_data.tuto3_folder() / "ID_109_red_16.npy")

We can define useful methods that implements raw images cleanup, implements sky subtractions and  perform cube stacking, to get our final processed image (*master*).

We can implement functions like the ones definded in 'image_pocesing.py'.

To inspect the **sky-backgroud** across on our image, we can define a function like:

 - `print_roi_mean_values()`:
  selects regions of interest (ROIs) of the image far from the science target and computes the mean pixel values within each region. These values represent the background noise in the image. Finally, the background is estimated as the avarage value across all the regions. 
    


In [5]:
def print_roi_mean_values(image, label=''):
    '''
    Print mean values of 4 Region Of Interest in the corners 
    of the image.
    Return average value in the ROIs
    We use it as background value indicator
    '''
    bkg_roi1 = image[50:100, 50:100].mean()
    bkg_roi2 = image[50:100, -100:-50].mean()
    bkg_roi3 = image[-100:-50, 50:100].mean()
    bkg_roi4 = image[-100:-50, -100:-50].mean()
    bkg = np.mean([bkg_roi1, bkg_roi2, bkg_roi3, bkg_roi4]) 
    print("%s : ROIs mean values %g %g %g %g - Average %g ADU" % (label, bkg_roi1, bkg_roi2, bkg_roi3, bkg_roi4, bkg) )
    return bkg

Once we check that the background across our image is constant, we get its **master** with a function like:
- `make_master_image()`: processes a raw data cube by subtracting a background image from each frame of the data cube and then combining the frames into a single master image.


In [ ]:
def make_master_image(raw_data_cube, background_image):
    ''' 
    Compute the master image from a raw cube data and the background image.
    
    1. Dark image is subtracted to each image of the raw cube
    2. Cube is cumulated (without shift and add)
    
    Dark_image must have the same camera setting
    (i.e. exposure and filter) of each image of the raw cube
    '''    
    
    Nframes = raw_data_cube.shape[-1]
    
    # create the background master 
    master_background = np.median(np.atleast_3d(background_image), axis = -1)

    
    # display the background image
    plt.figure()
    plt.imshow(master_background, vmin=0,vmax=2000)
    #plt.imshow(master_background)
    plt.colorbar(label = 'ADU')
    plt.title("Background image")
    print_roi_mean_values(master_background, label='Background')
 
    # display one frame of the raw data cube
    plt.figure()
    plt.imshow(raw_data_cube[:,:,0], vmin=0, vmax=2000)
    plt.colorbar(label = 'ADU')
    plt.title("Raw data image #0")
    print_roi_mean_values(raw_data_cube[:,:,0], label='Raw image #0')
    
    
    # subtracting background from raw data
    # make sure new array is float !!! to avoid integer overflows
    background_subtracted_data_cube = np.zeros(raw_data_cube.shape, dtype=float)
    for frame in np.arange(Nframes):
        background_subtracted_data_cube[:, :, frame] = raw_data_cube[ :, :, frame] - master_background
    

    ##########  Computing master image
    # Sum along the NDIT dimension to obtain a single (512,640) image
    # Must know total integration time
    # 
    # Advanced: shift & add. Detects every image's maximum and shift 
    # every image before stacking to eliminate residual tip-tilt
    ###########    
    master_image = background_subtracted_data_cube.sum(axis = -1)
    print_roi_mean_values(master_image, label='Master image')

    
    #########
    # Check background subtraction is ok.
    # Dark area should be around 0 
    ######
    plt.figure()
    plt.imshow(master_image, vmin=-10, vmax=100)
    plt.title('Master image (linear scale, clipped)')
    plt.colorbar()

    #########
    # Display image 
    # Dark area should be around 0 
    ######
    plt.figure()
    arr=master_image
    plt.imshow(np.log10(arr-np.median(arr)+1), cmap='inferno')
    plt.title('Master image (log scale)')
    plt.colorbar()
        
    return master_image

So with our defined function, we can proceed with subtracting the background from the open and closed data and get our master images.

In [6]:
## Reducing closed loop data
background_image = InfraredExampleData.get_camera_dark_data()
cl_raw_image_cube = InfraredExampleData.get_close_loop_data_cube()   
cl_master = make_master_image(cl_raw_image_cube, background_image)

In [7]:
## Reducing open loop data
ol_raw_image_cube = InfraredExampleData.get_open_loop_data_cube()    
ol_master = make_master_image(ol_raw_image_cube, background_image)

Now we can display our open loop and closed loop master images.

As a visual check, we can zoom into the PSF and remove negative values to display the images on a logarithmic scale.

In [8]:
ol_ima = ol_master[200:340, 290:430]
cl_ima = cl_master[200:340, 290:430]


fig, axs = plt.subplots(1, 2, sharex=True, sharey=True, figsize=(10, 5))  # Adjusting figure size for better layout
fig.suptitle('Long Exposure PSF')

# Plot the first image and its colorbar (Open Loop)
im0 = axs[0].imshow(np.log10(np.clip(ol_ima, 0, None) + 1), cmap='inferno')
axs[0].title.set_text('Open Loop')
fig.colorbar(im0, ax=axs[0])  # Add colorbar to the first plot

# Plot the second image and its colorbar (Close Loop)
im1 = axs[1].imshow(np.log10(np.clip(cl_ima, 0, None) + 1), cmap='inferno')
axs[1].title.set_text('Close Loop')
fig.colorbar(im1, ax=axs[1])  # Add colorbar to the second plot

fig.tight_layout(rect=[0, 0, 1, 0.95])  # Adjust layout so that title doesn't overlap with the subplots
plt.show()


## 1.3 Warnings:

Well, we went too easy so far :-) ! 

We need to keep in mind the following key aspects, that are crucial for noise subtraction and to improve the scientific return.

Among these, we find:

- **Image Subtraction**: when subtracting the background from the raw images they must be taken with the same exposure time.

- **Check Camera Saturation**: when integrating the raw images into a master image, it’s important to ensure that the camera has not saturated in the brighter regions of the science target. If the peak of the PSF becomes saturated, the FWHM will be underestimated.

- **Background inspection and estimation**: in the previous example, we assumed that the background was constant across the field. However, in real observations the background level can vary spatially due to factors such as uneven sky brightness, and it's important to account for this variation when performing background subtraction.

- **Detector artefact**: detectors often contain hot or cold pixels (pixels that exhibit spurius high or low signal levels), which may result in larger readout noise. These artefacts pollutes measurements and should be identified and corrected during image processing.

- **Shift & Add**: in image processing, it is used to improve image quality, particularly when combining multiple images into a data cube, by compensating for slight movements in the centroid of the PSF between successive images, which can occur due to atmospheric turbulence (i.e. Tip/Tilt) or telescope's tracking drifts.



**NB**: 

In this tutorial, our primary focus is on reducing background noise in images to evaluate the performance of AO-assisted observations. As mentioned in the introduction, performing scientific analysis on the final master image requires careful calibration using auxiliary images, such as bias, flat, and dark frames. 

Furthermore, it's important to note that, for scientific analysis, additional steps are required beyond background subtraction. For instance, if we were performig photometriy, we would need to convert the analog-to-digital units (ADU) of the science camera into physical flux units to quantify the brightness of objects. Similarly, if the goal is to perform astrometry, we would need to into account for the pixel scale and correct any distortions in the image to accurately measure objects' position and study their kinematics.

For a comprehensive discussion of these topics, see [(J. Heidt, 2022)](https://ui.adsabs.harvard.edu/abs/2022ASSL..467.....H/abstract).


## 1.4 Tasks

Now it's your turn :-) ! Can you improve the image reduction routine taking into account the above warnings?

Particularly, improve the reduction process by:

1. Checking if the integrated image is saturated.
2. Checking that the background is constant across the field. 
3. Detect and remove possible detector artefacts (such as hot or cold pixels). 
4. Apply Shift & Add before integrating the reduced image.